## STT Reports
This notebook uses an example analyst to generate some sample reports. Our analyst worked from February 1 to February 28, 2022; 7 days per week, ten hours per day, in two-hour increments (not unlike myself). For each two-hour increment, he worked on a randomly chosen project and activity. I'll begin the reports by looking at the projects and activities in the database.

In [46]:
import pandas as pd
import sqlite3
conn = sqlite3.connect("tests/data/stt.db")
sql = "select distinct project from records"
print(pd.read_sql_query(sql, conn));

     project
0  Project 0
1  Project 1
2  Project 2


In [47]:
sql = "select distinct activity from records";
print(pd.read_sql_query(sql,conn));

     activity
0  activity 0
1  activity 1
2  activity 2
3  activity 3
4  activity 4


### Report 1: Project Activities
Let's see how much time our analyst worked on Project 0. This is a simple query with one twist. The duration column in the records table is stored as total seconds. I'll leave it as is for now.

In [48]:
sql = "select project,activity, sum(duration) from records where project = 'Project 0' group by activity;"
df1 = pd.read_sql_query(sql, conn)
print(df1);

     project    activity  sum(duration)
0  Project 0  activity 0        50400.0
1  Project 0  activity 1        86400.0
2  Project 0  activity 2        72000.0
3  Project 0  activity 3        57600.0
4  Project 0  activity 4        93600.0


It would be trivial to make a similar report that selects an activity and groups by project. The more immediate problem is how to convert the duration column to something we can use. The read_sql_query function is able to parse dates, but not timedeltas. The solution is to do our own conversion. 

In [49]:
def convert_td(x):
    return x/3600;
df1["hours"] = df1["sum(duration)"].map(convert_td)
df1.drop(columns="sum(duration)", inplace=True)
print(df1)

     project    activity  hours
0  Project 0  activity 0   14.0
1  Project 0  activity 1   24.0
2  Project 0  activity 2   20.0
3  Project 0  activity 3   16.0
4  Project 0  activity 4   26.0


### Report 2: Project Time Per Activity
Here the focus is on how much time was recorded for each activity, aggregated by project.

In [50]:
def report_2():
    sql = "select activity, project, sum(duration) from records group by activity,project;"
    df = pd.read_sql_query(sql, conn);
    df["hours"] = df["sum(duration)"].map(convert_td)
    df.drop(columns="sum(duration)", inplace=True)
    return df;
    
df2 = report_2()
print(df2)

      activity    project  hours
0   activity 0  Project 0   14.0
1   activity 0  Project 1    6.0
2   activity 0  Project 2   14.0
3   activity 1  Project 0   24.0
4   activity 1  Project 1   14.0
5   activity 1  Project 2   22.0
6   activity 2  Project 0   20.0
7   activity 2  Project 1   26.0
8   activity 2  Project 2   20.0
9   activity 3  Project 0   16.0
10  activity 3  Project 1   20.0
11  activity 3  Project 2   12.0
12  activity 4  Project 0   26.0
13  activity 4  Project 1   22.0
14  activity 4  Project 2   24.0


Of course, it would be nice to see the total hours per activity.

In [51]:
def make_nice(df, grouper):
    groups = df.groupby(grouper)
    for name, group in groups:
        print(name.capitalize())
        print(group)
        print("---------------------------------")
        print(name.capitalize(),"total hours",sum(group['hours']))
        print("---------------------------------")
        
make_nice(df2, 'activity')

Activity 0
     activity    project  hours
0  activity 0  Project 0   14.0
1  activity 0  Project 1    6.0
2  activity 0  Project 2   14.0
---------------------------------
Activity 0 total hours 34.0
---------------------------------
Activity 1
     activity    project  hours
3  activity 1  Project 0   24.0
4  activity 1  Project 1   14.0
5  activity 1  Project 2   22.0
---------------------------------
Activity 1 total hours 60.0
---------------------------------
Activity 2
     activity    project  hours
6  activity 2  Project 0   20.0
7  activity 2  Project 1   26.0
8  activity 2  Project 2   20.0
---------------------------------
Activity 2 total hours 66.0
---------------------------------
Activity 3
      activity    project  hours
9   activity 3  Project 0   16.0
10  activity 3  Project 1   20.0
11  activity 3  Project 2   12.0
---------------------------------
Activity 3 total hours 48.0
---------------------------------
Activity 4
      activity    project  hours
12  activity

### Report 3: Most Recent Seven Days
Naturally we need the ability to filter by time period. This can be easily done for any time period in the database. Here we know that the most recent records are from February 28, 2022. This report shows activity for seven days up to and including that date.

In [52]:
def report_3():
    sql = "select project, activity, sum(duration) from records \
    where start > date('2022-02-28', '-6 days') group by project, activity;"
    df = pd.read_sql_query(sql, conn);
    df["hours"] = df["sum(duration)"].map(convert_td)
    df.drop(columns="sum(duration)", inplace=True)
    return df;
df3 = report_3();
make_nice(df3, 'project');
conn.close();

Project 0
     project    activity  hours
0  Project 0  activity 0    2.0
1  Project 0  activity 1    4.0
2  Project 0  activity 2    6.0
3  Project 0  activity 3    8.0
4  Project 0  activity 4    2.0
---------------------------------
Project 0 total hours 22.0
---------------------------------
Project 1
     project    activity  hours
5  Project 1  activity 1    4.0
6  Project 1  activity 2   10.0
7  Project 1  activity 3    8.0
8  Project 1  activity 4    6.0
---------------------------------
Project 1 total hours 28.0
---------------------------------
Project 2
      project    activity  hours
9   Project 2  activity 0    2.0
10  Project 2  activity 1    8.0
11  Project 2  activity 2    2.0
12  Project 2  activity 3    4.0
13  Project 2  activity 4    4.0
---------------------------------
Project 2 total hours 20.0
---------------------------------


### Conclusion
I have to give credit to pandas (and sqlite) for making this sort of thing super easy to do. It's only slightly more complicated to do the same type of thing with databases like MySql or PostgreSQL. My objectives for this project were to refresh my sql skills and experiment with the pandas read_sql capability. Both pandas and sqlite work fantastically well. Having completed this exercise, I have much bigger goals in mind for my next project.